In [1]:
!pip install -U torch torchvision --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB 20.5 MB/s eta 0:02:01
     ---------------------------------------- 0.0/2.5 GB 35.8 MB/s eta 0:01:09
     ---------------------------------------- 0.0/2.5 GB 35.4 MB/s eta 0:01:10
     ---------------------------------------- 0.0/2.5 GB 40.4 MB/s eta 0:01:02
     ---------------------------------------- 0.0/2.5 GB 41.6 MB/s eta 0:01:00
     ---------------------------------------- 0.0/2.5 GB 43.7 MB/s eta 0:00:57
     ---------------------------------------- 0.0/2.5 GB 43.7 MB/s eta 0:00:57
     ---------------------------------------- 0.0/2.5 GB 46.7 MB/s eta 0:00:53
     ---------------------------------------- 0.0/2.5 GB 46.7 MB/s eta 0:00:53
     ---------------------------------------- 0.0/2.5 GB 43.5 MB/s eta 0:00:57
     ---------------------------------------- 0.0/2.5 GB 43.5 MB/s eta 0:00:57
 


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Load Data
- Traverse train/test folders
- Create Xtrain/ytrain & Xtest/ytest vectors
- Load into matrix

In [2]:
# Encoding: 1 represents defective, 0 represents non-defective
"""
import os
from PIL import Image
import numpy as np

X_train = []
X_test = []
base_path = os.path.join("pepsico-lab-potato-quality-control", "Pepsico RnD Potato Lab Dataset")

for folder in ['Train', 'Test']:
    for category in ['Non-Defective', 'Defective']:
        folder_path = os.path.join(base_path, folder, category)
        for image_path in os.listdir(folder_path):
            if image_path.endswith('.jpg') is False:
                continue
            img = Image.open(os.path.join(folder_path, image_path))
            if folder == 'Train':
                X_train.append(np.asarray(img))
            else:
                X_test.append(np.asarray(img))

X_train = np.array(X_train)
X_test = np.array(X_test)

print(f'X_test Shape: {X_test.shape}')
print(f'X_train Shape: {X_train.shape}')

with open('X_train.npy', 'wb') as file:
    np.save(file, X_train)

with open('X_test.npy', 'wb') as file:
    np.save(file, X_test)
"""

'\nimport os\nfrom PIL import Image\nimport numpy as np\n\nX_train = []\nX_test = []\nbase_path = os.path.join("pepsico-lab-potato-quality-control", "Pepsico RnD Potato Lab Dataset")\n\nfor folder in [\'Train\', \'Test\']:\n    for category in [\'Non-Defective\', \'Defective\']:\n        folder_path = os.path.join(base_path, folder, category)\n        for image_path in os.listdir(folder_path):\n            if image_path.endswith(\'.jpg\') is False:\n                continue\n            img = Image.open(os.path.join(folder_path, image_path))\n            if folder == \'Train\':\n                X_train.append(np.asarray(img))\n            else:\n                X_test.append(np.asarray(img))\n\nX_train = np.array(X_train)\nX_test = np.array(X_test)\n\nprint(f\'X_test Shape: {X_test.shape}\')\nprint(f\'X_train Shape: {X_train.shape}\')\n\nwith open(\'X_train.npy\', \'wb\') as file:\n    np.save(file, X_train)\n\nwith open(\'X_test.npy\', \'wb\') as file:\n    np.save(file, X_test)\n'

In [36]:
import os
import random
import torch
from torch.utils.data import DataLoader, Subset
import torchvision
from torchvision import transforms
from lenet import LeNet
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.nn.functional import sigmoid
from PIL import Image
import numpy as np
import cv2

def bgremove3(myimage):
    # BG Remover 3
    myimage_hsv = cv2.cvtColor(myimage, cv2.COLOR_BGR2HSV)
     
    #Take S and remove any value that is less than half
    s = myimage_hsv[:,:,1]
    s = np.where(s < 80, 0, 1) # Any value below 127 will be excluded
 
    # We increase the brightness of the image and then mod by 255
    v = (myimage_hsv[:,:,2] + 80) % 255
    v = np.where(v > 80, 1, 0)  # Any value above 127 will be part of our mask
 
    # Combine our two masks based on S and V into a single "Foreground"
    foreground = np.where(s+v > 0, 1, 0).astype(np.uint8)  #Casting back into 8bit integer
 
    background = np.where(foreground==0,255,0).astype(np.uint8) # Invert foreground to get background in uint8
    background = cv2.cvtColor(background, cv2.COLOR_GRAY2BGR)  # Convert background back into BGR space
    foreground=cv2.bitwise_and(myimage,myimage,mask=foreground) # Apply our foreground map to original image
    finalimage = background+foreground # Combine foreground and background
    img_np = np.array(finalimage)
    return img_np
    #return finalimage

class Dataset(torch.utils.data.Dataset):
    def __init__(self, file_ids, labels, base_path, is_training):
        """
        Constructor for the Dataset.

        Parameters
        ----------
        file_ids : list[str]
            List of file names (not paths)
        labels : list[int]
            List of class identifiers, corresponding to the list of file IDs
        base_path : str
            Path to the folder containing the files in `file_ids`

        Returns
        -------
        Dataset
        """
        self.file_ids = file_ids
        self.labels = labels
        self.base_path = base_path
        transform_list = [
            transforms.ToPILImage(),
            transforms.Resize(28)
        ]
        if is_training:
            transform_list.extend([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(degrees=20),
                #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
            ])
        transform_list.append(transforms.ToTensor())
        self.transform = transforms.Compose(transform_list)
    
    def __len__(self):
        """
        Return the size of the dataset.

        Parameters
        ----------

        Returns
        -------
        int
        """
        return len(self.file_ids)

    def __getitem__(self, index):
        """
        Get a feature and label tuple based on the index.

        Parameters
        ----------
        index : int
            This function does not check the bounds of the Dataset

        Returns
        -------
        (torch.tensor, int) : Image vector and class
        """
        filename = self.file_ids[index]
        label = self.labels[index]
        img = torchvision.io.read_image(os.path.join(self.base_path, "Defective" if label == 1 else "Non-Defective", filename))
        return self.transform(img), self.labels[index]


def generate_dataset(is_train):
    """
    Generate training and testing datasets for the PepsiCo folder structure.

    Parameters
    ----------
    is_train: bool

    Returns
    -------
    Dataset
    """
    base_path = os.path.join("pepsico-lab-potato-quality-control", "Pepsico RnD Potato Lab Dataset")
    train_path = os.path.join(base_path, "Train" if is_train else "Test")
    file_ids = []
    labels = []
    for category in ["Defective", "Non-Defective"]:
        for filename in os.listdir(os.path.join(train_path, category)):
            if filename.endswith(".jpg") is False:
                continue
            file_ids.append(filename)
            labels.append(1 if category == 'Defective' else 0)
    return Dataset(file_ids, labels, train_path, is_train)

In [37]:
raw_train = generate_dataset(is_train=True)
raw_test = generate_dataset(is_train=False)

test_loader = DataLoader(raw_test, batch_size=32,shuffle=False,pin_memory=True)

raw_train_len = len(raw_train)
indices = random.sample(range(raw_train_len), raw_train_len)
train_indices = indices[:int(raw_train_len * 0.8)]
val_indices = list(set(indices).difference(set(train_indices)))
train = Subset(raw_train, train_indices)
validation = Subset(raw_train, val_indices)
train_loader = DataLoader(train, batch_size=32, shuffle=True, pin_memory=True)
validation_loader = DataLoader(validation, batch_size=32, shuffle=False, pin_memory=True)

In [38]:
model = LeNet(3, 1)
gpu = torch.device('cuda')
model = model.to(device=gpu)
loss = BCEWithLogitsLoss()
optimizer = Adam(model.parameters())

epochs = 10
for epoch in range(epochs):
    print(f'Epoch: {epoch}')
    # each batch contains 32 images
    current_batch = 0
    for batch in train_loader:
        if current_batch % 4 == 0:
            print(f'Batch: {current_batch}')
        current_batch += 1
        # batch[0] contains features
        X_batch, y_batch = batch[0].to(gpu), batch[1].to(gpu).unsqueeze(1).float()
        output = model.forward(X_batch)
        loss_value = loss(output, y_batch)
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
    print(f'Train Loss: {loss_value}')
    with torch.set_grad_enabled(False):
        correct = 0
        for val_batch in validation_loader:
            X_validation, y_validation = val_batch[0].to(gpu), val_batch[1].to(gpu).unsqueeze(1)
            val_preds = model.forward(X_validation)
            # print(sigmoid(val_preds))
            # print(y_validation)
            correct += int(sum((sigmoid(val_preds) >= 0.5) == y_validation))
        # Probability > 0.5 => Defective, otherwise non-defective
        # validation[1] (1 => Defective, 0 => Non-Defective)
        acc = correct * 100 / len(val_indices)
        print(f'Validation Accuracy: {acc:.2f}%')
    print()

with torch.set_grad_enabled(False):
    correct = 0
    for test_batch in test_loader:
        X_test, y_test = test_batch[0].to(gpu), test_batch[1].to(gpu).unsqueeze(1)
        test_preds = model.forward(X_test)
        # print(sigmoid(val_preds))
        # print(y_validation)
        correct += int(sum((sigmoid(test_preds) >= 0.5) == y_test))
    # Probability > 0.5 => Defective, otherwise non-defective
    # validation[1] (1 => Defective, 0 => Non-Defective)
    acc = correct * 100 / len(raw_test)
    print(f'Test Set Accuracy: {acc:.2f}%')

Epoch: 0
Batch: 0
Batch: 4
Batch: 8
Batch: 12
Batch: 16
Train Loss: 0.7066290974617004
Validation Accuracy: 50.00%

Epoch: 1
Batch: 0
Batch: 4
Batch: 8
Batch: 12
Batch: 16
Train Loss: 0.6478735208511353
Validation Accuracy: 85.06%

Epoch: 2
Batch: 0
Batch: 4
Batch: 8
Batch: 12
Batch: 16
Train Loss: 0.5622721314430237
Validation Accuracy: 92.21%

Epoch: 3
Batch: 0
Batch: 4
Batch: 8
Batch: 12
Batch: 16
Train Loss: 0.1508335918188095
Validation Accuracy: 94.16%

Epoch: 4
Batch: 0
Batch: 4
Batch: 8
Batch: 12
Batch: 16
Train Loss: 0.03377965837717056
Validation Accuracy: 94.81%

Epoch: 5
Batch: 0
Batch: 4
Batch: 8
Batch: 12
Batch: 16
Train Loss: 0.3381361663341522
Validation Accuracy: 96.75%

Epoch: 6
Batch: 0
Batch: 4
Batch: 8
Batch: 12
Batch: 16
Train Loss: 0.034604109823703766
Validation Accuracy: 93.51%

Epoch: 7
Batch: 0
Batch: 4
Batch: 8
Batch: 12
Batch: 16
Train Loss: 0.051191285252571106
Validation Accuracy: 95.45%

Epoch: 8
Batch: 0
Batch: 4
Batch: 8
Batch: 12
Batch: 16
Train Loss: